# Úkol č. 2 - předzpracování dat a binární klasifikace (do 7. prosince)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [1]:
import pandas as pd
import numpy as np
import math
from scipy.misc import logsumexp
import sklearn.metrics as metrics
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
pd.set_option('display.max_rows', 2000)

In [2]:
def upravaDat(data):
    display(data[data.drop(['cabin','age','home.dest'],axis=1).isnull().any(axis=1)])
    data = fare(data)
    data = embarked(data)    
    data = pclass(data)
    data = name(data)
    data = sex(data)    
    data = sibsp(data)
    data = parch(data)
    data = ticket(data)
    data = cabin(data)
    data = age(data)
    data = home_dest(data)
    return data

In [3]:
def pclass(data):
    print('\n-----------pclass-------\n')
    display(data['pclass'].value_counts())
    return data

In [4]:
def name(data):
    data.insert(3,'titul',data.iloc[:,3].str.split(' ').str[1])
    for i in data.index:
        if data.loc[i,'titul'] != 'Mr.' and data.loc[i,'titul'] != 'Miss.' and data.loc[i,'titul'] != 'Mrs.' and data.loc[i,'titul'] != 'Master.':
            data.loc[i,'titul'] = 'Other'
    data = data.drop(['name'],axis=1)
    data['titul'] = data['titul'].map({'Mr.' : 0 , 'Miss.' : 1 , 'Mrs.' : 2 , 'Master.' : 3 , 'Other' : 4 })
    return data

In [5]:
def sex(data):
    data['sex'] = data['sex'].map({'female' : 0 , 'male' : 1 })
    data['sex'] = data['sex'].astype('int64')
    return data

In [17]:
def age(data):
    print('\n-----------age-------\n')
    display(data.corr().age)
    
    for i in np.arange(1, 4, 1):
        for j in np.arange(0, 7, 1):
            data.loc[(data['age'].isnull()) & (data['pclass'] == i) & (data['sibsp'] == j), 'age'] = data[ (data['pclass']== i) & (data['sibsp'] == j) ]['age'].mean()
    
    data['age'] = pd.cut(data['age'].astype('int64'),bins=[0,10,18,25,30,45,55,1000],labels=[0,1,2,3,4,5,6],include_lowest=True)
    data['age'] = data['age'].astype('int64')
    display(data['age'].value_counts())
    return data

In [7]:
def sibsp(data):

    data.loc[data['sibsp'] > 4, 'sibsp'] = 5

    print('\n-----------sibsp-------\n')
    display(data['sibsp'].value_counts())
    return data

In [8]:
def parch(data):

    data.loc[data['parch'] > 4, 'parch'] = 5
    print('\n-----------parch-------\n')
    display(data['parch'].value_counts())
    return data

In [9]:
def ticket(data):
    data = data.drop(['ticket'],axis=1)
    return data

In [10]:
def fare(data):
    data['fare'] = data['fare'].fillna(data[data['pclass']==3]['fare'].value_counts().keys()[0])
    data['fare'] = pd.cut(data['fare'].astype('int64'),bins=[0,10,30,70,100,1000],labels=[0,1,2,3,4],include_lowest=True)
    data['fare'] = data['fare'].astype('int64')
    return data

In [11]:
def cabin(data):
    print('\n-----------cabin-------\n')
    data.insert(11,'paluba',data.loc[:,'cabin'].str[0])
    data.insert(11,'pozice',data.loc[:,'cabin'].str[1:].str.split(' ').str[0])
    data = data.drop(['cabin'],axis=1)
    for i in data.index:
        if data.loc[i,'pozice'] == '':
            data.loc[i,'pozice'] = np.nan
    data['pozice'] = data['pozice'].fillna(int(data[data.notnull().pozice]['pozice'].astype('int64').mean()))
    data['paluba'] = data['paluba'].fillna('C')    
    data.loc[data['paluba'] > 'F', 'paluba'] = 'G'
    display(data['paluba'].value_counts())
    data['paluba'] = data['paluba'].map({'A' : 0 , 'B' : 1 , 'C' : 2 , 'D' : 3 , 'E' : 4 , 'F' : 5 , 'G' : 6})
    
    data['pozice'] = pd.cut(data['pozice'].astype('int64'),bins=[0,20,45,50,100,1000],labels=[0,1,2,3,4],include_lowest=True)
    data['pozice'] = data['pozice'].astype('int64')
    
    display(data['pozice'].value_counts())
    
    return data

In [12]:
def embarked(data):
    data['embarked'] = data['embarked'].fillna(data['embarked'].value_counts().keys()[0])
    print('\n-----------embarked-------\n')
    display(data['embarked'].value_counts())
    data['embarked'] = data['embarked'].map({'S' : 1 , 'C' : 0 , 'Q' : 2})
    
    return data

In [13]:
def home_dest(data):
    data = data.drop(['home.dest'],axis=1)
    return data

In [15]:
def forest(data):
    X, X_test, Y, Y_test = train_test_split(data.drop(columns = ['ID', 'survived']), data['survived'], test_size=0.25, random_state=42)
    Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size=0.25, random_state=235)
    param_grid = {
        'n_estimators': range(1,100,5),
        'max_depth': range(1,6)
    }
    param_comb = ParameterGrid(param_grid)
    val_acc = []
    for param in param_comb:
        ret = RandomForestClassifier(**param)
        ret.fit(Xtrain, Ytrain)
        val_acc.append(math.sqrt(metrics.mean_squared_error(Yval, ret.predict(Xval))))
    best_param = param_comb[np.argmin(val_acc)]
    ret = RandomForestClassifier(**best_param)
    ret.fit(Xtrain, Ytrain)
    print('accuracy score (train): {0:.2f}'.format(metrics.accuracy_score(Y, ret.predict(X))*100),'%')
    print('accuracy score (test): {0:.2f}'.format(metrics.accuracy_score(Y_test, ret.predict(X_test))*100),'%')
    return ret

In [18]:
data = pd.read_csv('data.csv')
data.info()
data = upravaDat(data)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
ID           1000 non-null int64
survived     1000 non-null int64
pclass       1000 non-null int64
name         1000 non-null object
sex          1000 non-null object
age          796 non-null float64
sibsp        1000 non-null int64
parch        1000 non-null int64
ticket       1000 non-null object
fare         999 non-null float64
cabin        229 non-null object
embarked     999 non-null object
home.dest    575 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 101.6+ KB


,ID,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
484,484,0,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,NaN
833,833,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,"Cincinatti, OH"



-----------embarked-------



S    700
C    199
Q    101
Name: embarked, dtype: int64


-----------pclass-------



...

2    795
1     71
3     60
0     49
4     25
Name: pozice, dtype: int64


-----------age-------



ID         -0.040071
survived   -0.069022
pclass     -0.406653
titul      -0.093171
sex         0.056081
age         1.000000
sibsp      -0.255702
parch      -0.158004
fare        0.186121
embarked   -0.093545
pozice     -0.018537
paluba     -0.026705
Name: age, dtype: float64

3    263
4    246
2    210
1     93
0     74
5     67
6     47
Name: age, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
ID          1000 non-null int64
survived    1000 non-null int64
pclass      1000 non-null int64
titul       1000 non-null int64
sex         1000 non-null int64
age         1000 non-null int64
sibsp       1000 non-null int64
parch       1000 non-null int64
fare        1000 non-null int64
embarked    1000 non-null int64
pozice      1000 non-null int64
paluba      1000 non-null int64
dtypes: int64(12)
memory usage: 93.8 KB


In [19]:
evaluate = pd.read_csv('evaluation.csv')
evaluate.info()
evaluate = upravaDat(evaluate)
evaluate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 12 columns):
ID           309 non-null int64
pclass       309 non-null int64
name         309 non-null object
sex          309 non-null object
age          250 non-null float64
sibsp        309 non-null int64
parch        309 non-null int64
ticket       309 non-null object
fare         309 non-null float64
cabin        66 non-null object
embarked     308 non-null object
home.dest    170 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 29.0+ KB


,ID,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
13,1013,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,NaN



-----------embarked-------



S    216
C     71
Q     22
Name: embarked, dtype: int64


-----------pclass-------



...

3    266
1     21
0     11
4      7
2      4
Name: pozice, dtype: int64


-----------age-------



ID         -0.003960
pclass     -0.415529
titul            NaN
sex         0.088015
age         1.000000
sibsp      -0.213351
parch      -0.142492
fare        0.178566
embarked   -0.038165
pozice     -0.095439
paluba     -0.171701
Name: age, dtype: float64

4    81
2    74
3    72
5    28
1    24
0    19
6    11
Name: age, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 11 columns):
ID          309 non-null int64
pclass      309 non-null int64
titul       309 non-null int64
sex         309 non-null int64
age         309 non-null int64
sibsp       309 non-null int64
parch       309 non-null int64
fare        309 non-null int64
embarked    309 non-null int64
pozice      309 non-null int64
paluba      309 non-null int64
dtypes: int64(11)
memory usage: 26.6 KB


In [20]:
ret = forest(data)
result = pd.DataFrame(
    {'ID': evaluate['ID'],
     'predikce_preziti': ret.predict(evaluate.drop(columns = ['ID']))
    })
result.to_csv('result.csv', index=False)

accuracy score (train): 84.53 %
accuracy score (test): 81.60 %
